# Dog Breed Prediction
## https://www.kaggle.com/c/dog-breed-identification/overview
### CS 4662
### Spring 2020
### Alan Garcia
### Ryan Peralta


# ** In this file we are going to implement the CNN with a lower amount of filters and a greater dropout rate so that the overfitting problem can be solved. **


### Imports need to run our code for the Dog Breed project

In [1]:
import PIL.Image
from numpy import asarray
from PIL import Image

#Code so that the notebook is run through the GPU
import plaidml.keras
plaidml.keras.install_backend()

#from tensorflow.keras import backend
# "Sequential" model lets us to define a stack of neural network layers
from keras.models import Sequential

# import the "core" layers:
from keras.layers import Dense, Dropout, Activation, Flatten
import matplotlib.image as mpimg

# CNN
from keras.layers import Convolution2D, MaxPooling2D

# import some utilities to transform our data
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

Using TensorFlow backend.


### The next cell is needed to resize the images to standard and more usable size. It only needs to be run one time at the start of running this project, then it can be commented out for future test runs.

In [2]:
'''
import os
counter = 0
d = "C:/Users/rpera/Desktop/Spring 2020/CS 4662/Project Data/train"
for path in os.listdir(d):
    full_path = os.path.join(d, path)
    if os.path.isfile(full_path):
        print(full_path)
        img = Image.open(full_path).convert('L')
        maxsize = (128, 128)
        img = img.resize(maxsize, PIL.Image.ANTIALIAS)
        img.save('resized_photos/'+ str(counter) + '.jpg')
        counter += 1

'''

'\nimport os\ncounter = 0\nd = "C:/Users/rpera/Desktop/Spring 2020/CS 4662/Project Data/train"\nfor path in os.listdir(d):\n    full_path = os.path.join(d, path)\n    if os.path.isfile(full_path):\n        print(full_path)\n        img = Image.open(full_path).convert(\'L\')\n        maxsize = (128, 128)\n        img = img.resize(maxsize, PIL.Image.ANTIALIAS)\n        img.save(\'resized_photos/\'+ str(counter) + \'.jpg\')\n        counter += 1\n\n'

In [3]:
listForX = []
for x in range(0, 10222):
    test = mpimg.imread("C:/Users/rpera/Desktop/Spring 2020/CS 4662/CS4662-Project/resized_photos/"+str(x)+".jpg")
    # test = mpimg.imread("/Users/Alan/Desktop/Data_Science_2/final project/resized_photos/"+str(x)+".jpg")
    testing = (test.reshape(1, 16384))
    listForX.append(testing[0])

### Convert our existing datafram into X by converting the values from a list into an array through numpy

In [4]:
df = pd.DataFrame(listForX)
df.values.tolist()
X = np.asarray(listForX)

### Read the labels.csv into y

In [5]:
y = pd.read_csv("C:/Users/rpera/Desktop/Spring 2020/CS 4662/Project Data/labels.csv")
#y = pd.read_csv("/Users/Alan/Desktop/Data_Science_2/final project/labels.csv")

In [6]:
y.head

<bound method NDFrame.head of                                      id                     breed
0      000bec180eb18c7604dcecc8fe0dba07               boston_bull
1      001513dfcb2ffafc82cccf4d8bbaba97                     dingo
2      001cdf01b096e06d78e9e5112d419397                  pekinese
3      00214f311d5d2247d5dfe4fe24b2303d                  bluetick
4      0021f9ceb3235effd7fcde7f7538ed62          golden_retriever
...                                 ...                       ...
10217  ffd25009d635cfd16e793503ac5edef0                    borzoi
10218  ffd3f636f7f379c51ba3648a9ff8254f            dandie_dinmont
10219  ffe2ca6c940cddfee68fa3cc6c63213f                  airedale
10220  ffe5f6d8e2bff356e9482a80a6e29aac        miniature_pinscher
10221  fff43b07992508bc822f33d8ffd902ae  chesapeake_bay_retriever

[10222 rows x 2 columns]>

### Dropping the id column from y so that it only contains a vector of labels

In [7]:
y = y.drop(['id'], axis=1)
y['breed'] = pd.factorize(y['breed'])[0]
y = y.breed.tolist()

### Split X and y into train and test

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)

In [9]:
y_train = np_utils.to_categorical(y_train, 120)
y_test = np_utils.to_categorical(y_test, 120)

### Shaping X

In [10]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train.shape

(8177, 128, 128, 1)

### Creating our sequential model

In [11]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(128,128,1)))
print(model.output_shape) # -> (None, 26, 26, 32)

(None, 126, 126, 32)


### Adding the layers to our model

In [12]:
# more hidden layers:
model.add(Convolution2D(32, (3, 3), activation='relu'))
print(model.output_shape)

# Pooling Layer:
model.add(MaxPooling2D(pool_size=(2,2)))
print(model.output_shape)

# Dropout layer to avoid overfitting
model.add(Dropout(0.25)) 

(None, 124, 124, 32)
(None, 62, 62, 32)


In [13]:
# output Fully connected Dense layers:
model.add(Flatten())
print(model.output_shape)

model.add(Dense(128, activation='relu'))
print(model.output_shape)

model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))
print(model.output_shape)

(None, 123008)
(None, 128)
(None, 120)


In [14]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],
              optimizer='adam')

In [15]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 123008)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               15745152  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

### Shaping X and y

In [16]:
X_train.shape
y_train.shape

(8177, 120)

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss',  min_delta=0.01, patience=3)
fitted_model = model.fit(X_train, y_train, validation_split=0.25, batch_size=32, epochs=15, verbose=1)

Train on 6132 samples, validate on 2045 samples
Epoch 1/15
6132/6132 [==============================] - 138s 22ms/step - loss: 4.9064 - accuracy: 0.0096 - val_loss: 4.7860 - val_accuracy: 0.0112
Epoch 2/15
6132/6132 [==============================] - 139s 23ms/step - loss: 4.7815 - accuracy: 0.0148 - val_loss: 4.7713 - val_accuracy: 0.0117
Epoch 3/15
6132/6132 [==============================] - 140s 23ms/step - loss: 4.7639 - accuracy: 0.0144 - val_loss: 4.7565 - val_accuracy: 0.0186
Epoch 4/15
6132/6132 [==============================] - 138s 22ms/step - loss: 4.7186 - accuracy: 0.0214 - val_loss: 4.6847 - val_accuracy: 0.0269
Epoch 5/15
6132/6132 [==============================] - 149s 24ms/step - loss: 4.6587 - accuracy: 0.0199 - val_loss: 4.6544 - val_accuracy: 0.0249
Epoch 6/15
3488/6132 [================>.............] - ETA: 1:09 - loss: 4.6199 - accuracy: 0.0232

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

accuracy = fitted_model.history['acc']
#val_accuracy = history.history['val_acc']
loss = fitted_model.history['loss']
#val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b-', label='Training accuracy')
#plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r-', label='Training loss')
#plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
# plt.show()

### Predicting based off of our model

In [ ]:
y_predict = model.predict(X_test, verbose=1)
print (y_predict.shape)

In [ ]:
y_predict

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print('The accuracy is: ', score[1])

## We obtained an accuracy of 2.5%. This might be because we overfit our model.